# Хакатон "Ритмы продаж"

Вы когда-нибудь задумывались, почему цена на один и тот же товар может меняться? Или как огромный маркетплейс вроде Wildberries понимает, сколько товаров нужно заготовить на складе? В основе лежит сложный баланс между ценой и спросом: чем ниже цена, тем больше покупают. Но... насколько больше?
В этой задаче вам предстоит заглянуть в "машинное отделение" одного из крупнейших маркетплейсов и поработать с данными о продажах кроссовок. У вас будут реальные обезличенные данные о продажах, ценах и остатках тысяч моделей. Сможете ли вы разгадать их взаимосвязь и научиться предсказывать будущее?
Вам предстоит построить модель, которая сможет точно спрогнозировать, сколько единиц конкретного товара купят за один день при определённой цене.



Задание:

Постройте модель, которая для каждой пары товар-день в тестовом периоде предскажет точное количество проданных единиц товара.

Данные:<br>
train.parquet - история продаж за определенный период.<br>
test.parquet - данные за 14 дней, следующих сразу за обучающим набором.<br>
sample_submission.csv - файл-шаблон для отправки вашего решения.<br>
Файлы train.parquet и test.parquet содержат:<br>
nm_id - анонимный идентификатор товара.<br>
dt - дата.<br>
price - цена товара в этот день.<br>
is_promo - флаг участия товара в промо-акции.<br>
prev_leftovers - остаток товара на складе на начало дня.<br>
qty - количество проданных единиц, присутствует только в train. Это ваш таргет<br>


Метрика:
Качество вашего решения будет оцениваться с помощью метрики взвешенной MAE (wMAE). Так как в данных много дней с нулевыми продажами, мы усиливаем вклад дней, когда продажи были, ведь ошибка в них критичнее.
Веса определяются следующим образом:

$$
w_i =
\begin{cases}
1, & \text{если } y_i = 0 \\
7, & \text{если } y_i > 0
\end{cases}
$$


Итоговая формула метрики:
$$
\mathrm{MAE}=\frac{1}{n}\sum_{i=1}^{n}\lvert y_i-\hat{y}_i\rvert
$$


где $y_i$ — истинное значение продаж, а $\hat{y}_i$ — ваш прогноз.

**Примечание.** Вес для дней с продажами $w_{\text{pos}} = 7$ был выбран не случайно. Он рассчитан по обучающей выборке так, чтобы сбалансировать общую сумму весов для обоих классов, исходя из доли дней с продажами $p \approx 0.13$.




In [ ]:
import pandas as pd
import numpy as np

In [ ]:
TRAIN_PATH = "/Users/slvic/Downloads/train.parquet"
TEST_PATH = "/Users/slvic/Downloads/test.parquet"
SUB_PATH = "/Users/slvic/Downloads/sample_submission.csv"

train = pd.read_parquet(TRAIN_PATH)
test = pd.read_parquet(TEST_PATH)
sub = pd.read_csv(SUB_PATH)

print("train:", train.shape)
print("test :", test.shape)
print("sub  :", sub.shape)

display(train.head(3))
display(test.head(3))
display(sub.head(3))


In [ ]:
train["dt"] = pd.to_datetime(train["dt"], errors="coerce")
test["dt"]  = pd.to_datetime(test["dt"], errors="coerce")

def date_range_info(df, name):
    dmin = df["dt"].min()
    dmax = df["dt"].max()
    ndays = df["dt"].nunique()
    print(f"{name}:")
    print(f"  min dt: {dmin}")
    print(f"  max dt: {dmax}")
    print(f"  unique days: {ndays}")
    print(f"  rows: {len(df):,}")
    print()

date_range_info(train, "TRAIN")
date_range_info(test, "TEST")


train_max = train["dt"].max()
test_min  = test["dt"].min()

print("Continuity check:")
print("  train_max:", train_max)
print("  test_min :", test_min)
print("  gap (days):", (test_min - train_max).days)


test_days = sorted(test["dt"].dropna().unique())
print("\nTEST days count:", len(test_days))
print("First 5 test days:", test_days[:5])
print("Last  5 test days:", test_days[-5:])


**Промежуточные выводы по данным** <br>
В трейне дана история продаж с 04.07.2024 по 07.07.2025 (369 дней).<br>
В тесте дана история продаж с 08.07.2025 по 21.07.2025 (14 дней).<br>


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 120
plt.rcParams["font.size"] = 11

## 1. Разведочный анализ (Exploratory Deep Dive)

### 1.1 Профилирование таргета
Распределение qty, доля нулей, выбросы

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].hist(train["qty"], bins=50, edgecolor="black")
axes[0].set_title("Распределение qty (все)")
axes[0].set_xlabel("qty")
axes[0].set_ylabel("Частота")

qty_pos = train.loc[train["qty"] > 0, "qty"]
axes[1].hist(qty_pos, bins=50, edgecolor="black", color="orange")
axes[1].set_title(f"Распределение qty > 0 (n={len(qty_pos):,})")
axes[1].set_xlabel("qty")

zero_rate = (train["qty"] == 0).mean()
axes[2].bar(["qty = 0", "qty > 0"], [zero_rate, 1 - zero_rate],
            color=["steelblue", "orange"])
axes[2].set_title(f"Доля нулей: {zero_rate:.1%}")
axes[2].set_ylabel("Доля")

plt.tight_layout()
plt.show()

print("Статистики qty:")
print(train["qty"].describe())
print(f"\nПерцентили qty > 0:")
print(qty_pos.describe(percentiles=[0.5, 0.75, 0.9, 0.95, 0.99]))

### 1.2 Временные паттерны
Средние продажи по дням недели, месяцам; тренд

In [ ]:
train["dow"] = train["dt"].dt.dayofweek
train["month"] = train["dt"].dt.month

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# По дням недели
dow_names = ["Пн", "Вт", "Ср", "Чт", "Пт", "Сб", "Вс"]
dow_stats = train.groupby("dow")["qty"].mean()
axes[0].bar(dow_names, dow_stats.values, color="steelblue")
axes[0].set_title("Средние продажи по дням недели")
axes[0].set_ylabel("mean qty")

# По месяцам
month_stats = train.groupby("month")["qty"].mean()
axes[1].bar(month_stats.index, month_stats.values, color="coral")
axes[1].set_title("Средние продажи по месяцам")
axes[1].set_xlabel("Месяц")
axes[1].set_ylabel("mean qty")

# Тренд: средние продажи по дням
daily = train.groupby("dt")["qty"].mean().reset_index()
axes[2].plot(daily["dt"], daily["qty"], alpha=0.5, linewidth=0.8)
axes[2].set_title("Средние дневные продажи (тренд)")
axes[2].tick_params(axis="x", rotation=45)

plt.tight_layout()
plt.show()

# Доля ненулевых продаж по дням недели
sell_rate_dow = train.groupby("dow")["qty"].apply(lambda x: (x > 0).mean())
print("Доля дней с продажами по дням недели:")
for i, d in enumerate(dow_names):
    print(f"  {d}: {sell_rate_dow.iloc[i]:.2%}")

### 1.3 Ценовая эластичность и промо-эффект
Связь цены со спросом; влияние промо на продажи

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Цена vs qty (scatter на сэмпле)
sample = train.sample(min(10_000, len(train)), random_state=42)
axes[0].scatter(sample["price"], sample["qty"], alpha=0.15, s=5)
axes[0].set_title("Цена vs Продажи")
axes[0].set_xlabel("price")
axes[0].set_ylabel("qty")

# Промо vs не-промо
promo_stats = train.groupby("is_promo")["qty"].agg(["mean", "sum", "count"])
promo_stats["sell_rate"] = train.groupby("is_promo")["qty"].apply(lambda x: (x > 0).mean()).values
print("Статистики по промо:")
display(promo_stats)

labels = ["Без промо", "С промо"]
axes[1].bar(labels, promo_stats["mean"].values, color=["steelblue", "orange"])
axes[1].set_title("Средние продажи: промо vs обычные")
axes[1].set_ylabel("mean qty")

axes[2].bar(labels, promo_stats["sell_rate"].values, color=["steelblue", "orange"])
axes[2].set_title("Доля дней с продажами: промо vs обычные")
axes[2].set_ylabel("sell rate")

plt.tight_layout()
plt.show()

### 1.4 Анализ остатков на складе
Связь prev_leftovers → qty; порог дефицита

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Остатки vs qty
axes[0].scatter(sample["prev_leftovers"], sample["qty"], alpha=0.15, s=5)
axes[0].set_title("Остатки vs Продажи")
axes[0].set_xlabel("prev_leftovers")
axes[0].set_ylabel("qty")

# Нулевые остатки = нулевые продажи?
zero_stock = train[train["prev_leftovers"] == 0]
print(f"Строк с нулевыми остатками: {len(zero_stock):,} ({len(zero_stock)/len(train):.2%})")
print(f"Из них с qty > 0: {(zero_stock['qty'] > 0).sum()} ({(zero_stock['qty'] > 0).mean():.2%})")

# Бины остатков
train["stock_bin"] = pd.cut(train["prev_leftovers"],
                            bins=[0, 5, 20, 50, 100, 500, float("inf")],
                            labels=["1-5", "6-20", "21-50", "51-100", "101-500", "500+"])
stock_agg = train[train["prev_leftovers"] > 0].groupby("stock_bin", observed=True)["qty"].agg(["mean", "count"])
stock_agg["sell_rate"] = train[train["prev_leftovers"] > 0].groupby("stock_bin", observed=True)["qty"].apply(lambda x: (x > 0).mean()).values

axes[1].bar(stock_agg.index.astype(str), stock_agg["mean"], color="teal")
axes[1].set_title("Средние продажи по бинам остатков")
axes[1].set_xlabel("prev_leftovers bin")
axes[1].set_ylabel("mean qty")

axes[2].bar(stock_agg.index.astype(str), stock_agg["sell_rate"], color="teal")
axes[2].set_title("Доля дней с продажами по бинам остатков")
axes[2].set_ylabel("sell rate")

plt.tight_layout()
plt.show()

train.drop(columns=["stock_bin"], inplace=True)

### 1.5 Товарная сегментация
Кластеры товаров по объёму продаж; пересечение товаров train ↔ test

In [ ]:
item_stats = train.groupby("nm_id").agg(
    total_qty=("qty", "sum"),
    mean_qty=("qty", "mean"),
    sell_rate=("qty", lambda x: (x > 0).mean()),
    n_days=("dt", "nunique"),
    avg_price=("price", "mean"),
).reset_index()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].hist(item_stats["total_qty"], bins=50, edgecolor="black")
axes[0].set_title("Распределение суммарных продаж по товарам")
axes[0].set_xlabel("total qty")

axes[1].hist(item_stats["sell_rate"], bins=50, edgecolor="black", color="orange")
axes[1].set_title("Распределение sell_rate по товарам")
axes[1].set_xlabel("sell rate (доля дней с продажами)")

axes[2].hist(item_stats["avg_price"], bins=50, edgecolor="black", color="teal")
axes[2].set_title("Распределение средней цены по товарам")
axes[2].set_xlabel("avg price")

plt.tight_layout()
plt.show()

# Пересечение товаров
train_items = set(train["nm_id"].unique())
test_items = set(test["nm_id"].unique())
overlap = train_items & test_items
only_train = train_items - test_items
only_test = test_items - train_items

print(f"Товаров в train: {len(train_items)}")
print(f"Товаров в test:  {len(test_items)}")
print(f"Пересечение:     {len(overlap)}")
print(f"Только в train:  {len(only_train)}")
print(f"Только в test:   {len(only_test)}")

# Топ-10 товаров по продажам
top10 = item_stats.nlargest(10, "total_qty")[["nm_id", "total_qty", "mean_qty", "sell_rate", "avg_price"]]
print("\nТоп-10 товаров по суммарным продажам:")
display(top10)

---
## 2. Предобработка и очистка (Data Sanitization)
- Обработка аномалий и выбросов
- Фильтрация "мёртвых" товаров (нулевые остатки)
- Объединение train и test для единого feature engineering

In [ ]:
# Перечитаем чистые данные для пайплайна
train = pd.read_parquet(TRAIN_PATH)
test = pd.read_parquet(TEST_PATH)
sub = pd.read_csv(SUB_PATH)

train["dt"] = pd.to_datetime(train["dt"])
test["dt"] = pd.to_datetime(test["dt"])

# Помечаем источник
train["is_test"] = 0
test["is_test"] = 1
test["qty"] = np.nan

# Объединяем
df = pd.concat([train, test], ignore_index=True)
df = df.sort_values(["nm_id", "dt"]).reset_index(drop=True)

print(f"Объединённый датасет: {df.shape}")
print(f"Train строк: {(df['is_test'] == 0).sum():,}")
print(f"Test строк:  {(df['is_test'] == 1).sum():,}")

In [ ]:
# ============================================================
# 2.x Удаление выбросов в train/val и логарифмирование при необходимости
# ============================================================
# Логика:
#  - Выбросы: значения числовых признаков, строго превышающие 0.996-квантиль (по train-части)
#  - Удаляем такие строки ТОЛЬКО в train-части (включая будущую валидацию). Тест не трогаем.
#  - Если после удаления выбросов по ящику с усами (IQR) у цены всё ещё есть выбросы, добавляем лог-признак для цены.

num_cols = []
for c in ["qty", "price", "prev_leftovers"]:
    if c in df.columns:
        num_cols.append(c)

train_mask = df["is_test"] == 0
removed_info = {}

# 0.996-квантили считаем по train-части
quantiles_996 = {c: df.loc[train_mask, c].quantile(0.996) for c in num_cols}

# Мастер-маска: строка помечается к удалению, если в ЛЮБОМ числе > квантиля
remove_mask = train_mask.copy() & False
for c in num_cols:
    thr = quantiles_996[c]
    over_thr = train_mask & (df[c] > thr)  # строго > квантиля
    removed_info[c] = int(over_thr.sum())
    remove_mask = remove_mask | over_thr

removed_total = int(remove_mask.sum())
if removed_total > 0:
    df = df.loc[~remove_mask].reset_index(drop=True)

print("0.996-квантили (train):")
for c in num_cols:
    print(f"  {c}: {quantiles_996[c]:.6g}")
print(f"Удалено строк суммарно (train/val): {removed_total:,}")
print("Вклад по столбцам (строк, где признак > квантиля):")
for c in num_cols:
    print(f"  {c}: {removed_info[c]:,}")

# Проверка по IQR для цены: если остались выбросы — добавим лог-цену
if "price" in df.columns:
    s = df.loc[df["is_test"] == 0, "price"].astype(float)
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
    has_outliers_price = bool(((s < lower) | (s > upper)).any())
    if has_outliers_price:
        # Логарифмируем цену (добавляем отдельный признак, базовую цену не трогаем)
        df["log_price"] = np.log1p(df["price"])
        print("По IQR для 'price' остались выбросы — добавлен признак log_price.")
    else:
        print("По IQR для 'price' выбросов не осталось.")

# Для prev_leftovers лог-признак создаётся далее (log_leftovers),
# поэтому отдельно ничего не делаем здесь.


---
## 3. Конструирование признаков (Feature Engineering)
Ключевой этап пайплайна. Создаём признаки на объединённом датасете, чтобы лаги корректно "перетекали" из train в test.

### 3.1 Календарные признаки
### 3.2 Лаговые и скользящие признаки (per-item)
### 3.3 Ценовые признаки
### 3.4 Признаки остатков
### 3.5 Агрегатные признаки товара (item-level statics)
### 3.6 Тренд-признаки

In [ ]:
# ============================================================
# 3.1 Календарные признаки
# ============================================================
df["dow"] = df["dt"].dt.dayofweek          # 0=Пн, 6=Вс
df["is_weekend"] = (df["dow"] >= 5).astype(int)
df["day_of_month"] = df["dt"].dt.day
df["week_of_year"] = df["dt"].dt.isocalendar().week.astype(int)
df["month"] = df["dt"].dt.month
df["day_of_year"] = df["dt"].dt.dayofyear

print("Календарные признаки добавлены:", ["dow", "is_weekend", "day_of_month", "week_of_year", "month", "day_of_year"])

In [ ]:
# ============================================================
# 3.2 Лаговые и скользящие признаки (per-item)
# ============================================================
# Важно: лаги считаем ТОЛЬКО по train-части qty (NaN в test не портит)

df = df.sort_values(["nm_id", "dt"]).reset_index(drop=True)

lag_days = [1, 2, 3, 7, 14, 28]
rolling_windows = [3, 7, 14, 28]

for lag in lag_days:
    df[f"qty_lag_{lag}"] = df.groupby("nm_id")["qty"].shift(lag)

for w in rolling_windows:
    rolled = df.groupby("nm_id")["qty"].shift(1).groupby(df["nm_id"]).rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)
    df[f"qty_rmean_{w}"] = rolled

for w in rolling_windows:
    rolled = df.groupby("nm_id")["qty"].shift(1).groupby(df["nm_id"]).rolling(w, min_periods=1).std().reset_index(level=0, drop=True)
    df[f"qty_rstd_{w}"] = rolled

# EWM
ewm_spans = [7, 14]
for span in ewm_spans:
    shifted = df.groupby("nm_id")["qty"].shift(1)
    ewm_val = shifted.groupby(df["nm_id"]).apply(lambda x: x.ewm(span=span, min_periods=1).mean()).reset_index(level=0, drop=True)
    df[f"qty_ewm_{span}"] = ewm_val

# Скользящая доля ненулевых продаж
for w in [7, 14, 28]:
    shifted = df.groupby("nm_id")["qty"].shift(1)
    sell_flag = (shifted > 0).astype(float)
    sr = sell_flag.groupby(df["nm_id"]).rolling(w, min_periods=1).mean().reset_index(level=0, drop=True)
    df[f"sell_rate_{w}"] = sr

lag_cols = [c for c in df.columns if "qty_lag" in c or "qty_rmean" in c or "qty_rstd" in c or "qty_ewm" in c or "sell_rate_" in c]

# === FORWARD-FILL: заполняем NaN в лаговых фичах последним известным значением ===
# Без этого 93% лаговых фичей в test = NaN (т.к. qty в test = NaN)
print("NaN в лаговых фичах ДО forward-fill (test):")
test_mask = df["is_test"] == 1
print(df.loc[test_mask, lag_cols].isnull().sum().sort_values(ascending=False).head(5))

for col in lag_cols:
    df[col] = df.groupby("nm_id")[col].ffill()

print(f"\nNaN в лаговых фичах ПОСЛЕ forward-fill (test):")
print(df.loc[test_mask, lag_cols].isnull().sum().sort_values(ascending=False).head(5))

print(f"\nЛаговых/скользящих признаков: {len(lag_cols)}")
print(lag_cols)

In [ ]:
# ============================================================
# 3.3 Ценовые признаки
# ============================================================

# Средняя цена товара (считаем только по train)
train_mask = df["is_test"] == 0
item_avg_price = df.loc[train_mask].groupby("nm_id")["price"].mean().rename("item_avg_price")
df = df.merge(item_avg_price, on="nm_id", how="left")

# Отношение текущей цены к средней (> 1 = дороже обычного)
df["price_ratio"] = df["price"] / df["item_avg_price"]
df["price_ratio"] = df["price_ratio"].fillna(1.0)

# Изменение цены за 1 день
df["price_diff_1"] = df.groupby("nm_id")["price"].diff(1)

# Скользящее среднее цены за 7 дней
df["price_rmean_7"] = (
    df.groupby("nm_id")["price"]
    .shift(1)
    .groupby(df["nm_id"])
    .rolling(7, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)
df["price_vs_rmean7"] = df["price"] / df["price_rmean_7"]
df["price_vs_rmean7"] = df["price_vs_rmean7"].fillna(1.0)

# Глубина скидки (если промо)
df["discount_depth"] = (df["item_avg_price"] - df["price"]) / df["item_avg_price"]
df["discount_depth"] = df["discount_depth"].clip(-1, 1)

price_feats = ["price_ratio", "price_diff_1", "price_rmean_7", "price_vs_rmean7", "discount_depth", "item_avg_price"]
print(f"Ценовых признаков: {len(price_feats)}")
print(price_feats)

In [ ]:
# ============================================================
# 3.4 Признаки остатков
# ============================================================

# Лог-остатки (для нелинейной связи)
df["log_leftovers"] = np.log1p(df["prev_leftovers"])

# Низкий остаток (может ограничивать продажи)
df["low_stock"] = (df["prev_leftovers"] <= 5).astype(int)
df["zero_stock"] = (df["prev_leftovers"] == 0).astype(int)

# Дней запаса (prev_leftovers / среднее qty за 7 дней)
df["days_of_stock"] = df["prev_leftovers"] / df["qty_rmean_7"].replace(0, np.nan)
df["days_of_stock"] = df["days_of_stock"].clip(0, 1000).fillna(1000)

# Изменение остатков за 1 день
df["leftovers_diff_1"] = df.groupby("nm_id")["prev_leftovers"].diff(1)

stock_feats = ["log_leftovers", "low_stock", "zero_stock", "days_of_stock", "leftovers_diff_1"]
print(f"Признаков остатков: {len(stock_feats)}")
print(stock_feats)

In [ ]:
# ============================================================
# 3.5 Агрегатные признаки товара (item-level statics)
# ============================================================
# Считаем ТОЛЬКО по train-части, чтобы не было утечки

train_part = df[df["is_test"] == 0]

item_agg = train_part.groupby("nm_id").agg(
    item_total_qty=("qty", "sum"),
    item_mean_qty=("qty", "mean"),
    item_median_qty=("qty", "median"),
    item_max_qty=("qty", "max"),
    item_std_qty=("qty", "std"),
    item_sell_rate=("qty", lambda x: (x > 0).mean()),
    item_n_days=("dt", "nunique"),
    item_price_std=("price", "std"),
    item_mean_leftovers=("prev_leftovers", "mean"),
    item_promo_rate=("is_promo", "mean"),
).reset_index()

item_agg["item_std_qty"] = item_agg["item_std_qty"].fillna(0)

df = df.merge(item_agg, on="nm_id", how="left")

# "Возраст" товара: сколько дней от первого появления
item_first = train_part.groupby("nm_id")["dt"].min().rename("item_first_dt")
df = df.merge(item_first, on="nm_id", how="left")
df["item_age"] = (df["dt"] - df["item_first_dt"]).dt.days
df.drop(columns=["item_first_dt"], inplace=True)

item_feats = [c for c in item_agg.columns if c != "nm_id"] + ["item_age"]
print(f"Агрегатных признаков товара: {len(item_feats)}")
print(item_feats)

In [ ]:
# ============================================================
# 3.6 Тренд-признаки
# ============================================================

# Отношение средних продаж за последнюю неделю к предпоследней
df["qty_ratio_w1_w2"] = df["qty_rmean_7"] / df["qty_rmean_14"].replace(0, np.nan)
df["qty_ratio_w1_w2"] = df["qty_ratio_w1_w2"].fillna(1.0).clip(0, 10)

# Отношение средних продаж за последние 7 дней к последним 28 дням
df["qty_ratio_w1_m1"] = df["qty_rmean_7"] / df["qty_rmean_28"].replace(0, np.nan)
df["qty_ratio_w1_m1"] = df["qty_ratio_w1_m1"].fillna(1.0).clip(0, 10)

# === Индикатор устаревшости лагов ===
# Для test-строк: сколько дней прошло с конца train → модель знает, что лаги «замороженные»
train_max_dt = df.loc[df["is_test"] == 0, "dt"].max()
df["days_since_train"] = (df["dt"] - train_max_dt).dt.days.clip(lower=0)

trend_feats = ["qty_ratio_w1_w2", "qty_ratio_w1_m1", "days_since_train"]
print(f"Тренд-признаков: {len(trend_feats)}")
print(trend_feats)

# ============================================================
# Финальный набор фичей
# ============================================================
feature_cols = (
    ["price", "is_promo", "prev_leftovers"]
    + ["dow", "is_weekend", "day_of_month", "week_of_year", "month", "day_of_year"]
    + lag_cols
    + price_feats
    + stock_feats
    + item_feats
    + trend_feats
)

print(f"\nИтого признаков: {len(feature_cols)}")
print(f"\nПроверка NaN в test (топ-10):")
test_part = df[df["is_test"] == 1]
nan_counts = test_part[feature_cols].isnull().sum().sort_values(ascending=False)
print(nan_counts[nan_counts > 0].head(10) if (nan_counts > 0).any() else "NaN нет!")

In [ ]:
# Добавим log_price в список признаков при наличии
if "log_price" in df.columns and "log_price" not in feature_cols:
    feature_cols = ["log_price"] + feature_cols
print("log_price in feature_cols:", "log_price" in feature_cols)

---
## 4. Стратегия валидации (Validation Design)
- **Time-based holdout**: последние 14 дней train как валидация (имитация теста)
- **Кастомная метрика wMAE**: w=7 для qty>0, w=1 для qty=0
- Исключаем первые ~28 дней из-за NaN в лаговых признаках

In [ ]:
# ============================================================
# Кастомная метрика wMAE
# ============================================================
def weighted_mae(y_true, y_pred, w_pos=7, w_zero=1):
    """Взвешенная MAE: вес w_pos для y>0, вес w_zero для y=0."""
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    weights = np.where(y_true > 0, w_pos, w_zero)
    return np.average(np.abs(y_true - y_pred), weights=weights)

# ============================================================
# Train / Validation split (time-based)
# ============================================================
train_df = df[df["is_test"] == 0].copy()
test_df = df[df["is_test"] == 1].copy()

# Валидация = последние 14 дней train
val_cutoff = train_df["dt"].max() - pd.Timedelta(days=13)
print(f"Val cutoff: {val_cutoff.date()}  →  val period: {val_cutoff.date()} - {train_df['dt'].max().date()}")

# Исключаем первые 28 дней (NaN в лагах)
train_start = train_df["dt"].min() + pd.Timedelta(days=28)
print(f"Train start (после прогрева лагов): {train_start.date()}")

tr = train_df[(train_df["dt"] >= train_start) & (train_df["dt"] < val_cutoff)].copy()
val = train_df[train_df["dt"] >= val_cutoff].copy()

print(f"\nTrain: {len(tr):,} строк, {tr['dt'].min().date()} — {tr['dt'].max().date()}")
print(f"Val:   {len(val):,} строк, {val['dt'].min().date()} — {val['dt'].max().date()}")

X_tr = tr[feature_cols]
y_tr = tr["qty"]
X_val = val[feature_cols]
y_val = val["qty"]
X_test = test_df[feature_cols]

# Веса для обучения
w_tr = np.where(y_tr > 0, 7.0, 1.0)
w_val = np.where(y_val > 0, 7.0, 1.0)

print(f"\nX_tr: {X_tr.shape}, X_val: {X_val.shape}, X_test: {X_test.shape}")

---
## 5. Моделирование (Modeling)

### 5.1 Baseline — среднее за последние 7 дней
### 5.2 LightGBM с sample_weight
### 5.3 CatBoost с sample_weight
### 5.4 Two-stage: классификация (есть продажа?) + регрессия (сколько?)
### 5.5 Ансамбль моделей

In [ ]:
# ============================================================
# 5.1 Baseline: среднее за последние 7 дней (qty_rmean_7)
# ============================================================
baseline_pred = val["qty_rmean_7"].fillna(0).values
baseline_wmae = weighted_mae(y_val, baseline_pred)
baseline_mae = np.mean(np.abs(y_val - baseline_pred))

print(f"Baseline (rmean_7):")
print(f"  MAE:  {baseline_mae:.4f}")
print(f"  wMAE: {baseline_wmae:.4f}")

In [ ]:
# ============================================================
# 5.2 LightGBM
# ============================================================
import lightgbm as lgb

lgb_params = {
    "objective": "regression_l1",  # MAE loss (ближе к метрике)
    "metric": "mae",
    "learning_rate": 0.05,
    "num_leaves": 63,
    "max_depth": -1,
    "min_child_samples": 30,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 0.1,
    "reg_lambda": 1.0,
    "n_estimators": 3000,
    "random_state": 42,
    "verbose": -1,
    "n_jobs": -1,
}

lgb_model = lgb.LGBMRegressor(**lgb_params)
lgb_model.fit(
    X_tr, y_tr,
    sample_weight=w_tr,
    eval_set=[(X_val, y_val)],
    eval_sample_weight=[w_val],
    callbacks=[
        lgb.early_stopping(100, verbose=True),
        lgb.log_evaluation(200),
    ],
)

lgb_val_pred = lgb_model.predict(X_val)
lgb_val_pred = np.clip(lgb_val_pred, 0, None)

print(f"\nLightGBM:")
print(f"  MAE:  {np.mean(np.abs(y_val - lgb_val_pred)):.4f}")
print(f"  wMAE: {weighted_mae(y_val, lgb_val_pred):.4f}")
print(f"  Best iteration: {lgb_model.best_iteration_}")

In [ ]:
# Feature importance (LightGBM)
fi = pd.DataFrame({
    "feature": feature_cols,
    "importance": lgb_model.feature_importances_
}).sort_values("importance", ascending=False)

fig, ax = plt.subplots(figsize=(10, 10))
ax.barh(fi["feature"].head(25)[::-1], fi["importance"].head(25)[::-1])
ax.set_title("LightGBM — Top-25 Feature Importance")
plt.tight_layout()
plt.show()

display(fi.head(15))

In [ ]:
# ============================================================
# 5.3 CatBoost (с взвешенным eval через Pool)
# ============================================================
from catboost import CatBoostRegressor, Pool

cb_train_pool = Pool(X_tr, y_tr, weight=w_tr)
cb_val_pool = Pool(X_val, y_val, weight=w_val)

cb_model = CatBoostRegressor(
    loss_function="MAE",
    iterations=3000,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=3.0,
    subsample=0.8,
    random_seed=42,
    early_stopping_rounds=100,
    verbose=200,
)

cb_model.fit(cb_train_pool, eval_set=cb_val_pool)

cb_val_pred = cb_model.predict(X_val)
cb_val_pred = np.clip(cb_val_pred, 0, None)

print(f"\nCatBoost:")
print(f"  MAE:  {np.mean(np.abs(y_val - cb_val_pred)):.4f}")
print(f"  wMAE: {weighted_mae(y_val, cb_val_pred):.4f}")
print(f"  Best iteration: {cb_model.get_best_iteration()}")

In [ ]:
# ============================================================
# 5.4 Two-stage: классификация + регрессия
# ============================================================

# Stage 1: Будет ли продажа? (binary classification)
y_tr_cls = (y_tr > 0).astype(int)
y_val_cls = (y_val > 0).astype(int)

cls_model = lgb.LGBMClassifier(
    objective="binary",
    metric="binary_logloss",
    learning_rate=0.05,
    num_leaves=63,
    min_child_samples=30,
    subsample=0.8,
    colsample_bytree=0.8,
    n_estimators=2000,
    random_state=42,
    verbose=-1,
    n_jobs=-1,
    scale_pos_weight=7.0,  # баланс классов
)
cls_model.fit(
    X_tr, y_tr_cls,
    eval_set=[(X_val, y_val_cls)],
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(500)],
)

cls_proba = cls_model.predict_proba(X_val)[:, 1]

# Stage 2: Регрессия только на ненулевых
tr_pos = tr[y_tr > 0]
X_tr_pos = tr_pos[feature_cols]
y_tr_pos = tr_pos["qty"]

reg_model = lgb.LGBMRegressor(
    objective="regression_l1",
    learning_rate=0.05,
    num_leaves=63,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    n_estimators=2000,
    random_state=42,
    verbose=-1,
    n_jobs=-1,
)

val_pos = val[y_val > 0]
X_val_pos = val_pos[feature_cols]
y_val_pos = val_pos["qty"]

reg_model.fit(
    X_tr_pos, y_tr_pos,
    eval_set=[(X_val_pos, y_val_pos)],
    callbacks=[lgb.early_stopping(100), lgb.log_evaluation(500)],
)

# Комбинируем: P(sale) * E[qty | sale]
reg_pred_all = reg_model.predict(X_val)
reg_pred_all = np.clip(reg_pred_all, 0, None)

# Подбираем порог для классификации
from sklearn.metrics import f1_score

best_thr, best_wmae_2s = 0.5, float("inf")
for thr in np.arange(0.05, 0.95, 0.05):
    pred_2s = np.where(cls_proba >= thr, reg_pred_all, 0.0)
    wm = weighted_mae(y_val, pred_2s)
    if wm < best_wmae_2s:
        best_thr = thr
        best_wmae_2s = wm

two_stage_pred = np.where(cls_proba >= best_thr, reg_pred_all, 0.0)

print(f"\nTwo-stage (threshold={best_thr:.2f}):")
print(f"  MAE:  {np.mean(np.abs(y_val - two_stage_pred)):.4f}")
print(f"  wMAE: {best_wmae_2s:.4f}")

In [ ]:
# ============================================================
# 5.5 Ансамбль — подбор весов на валидации
# ============================================================
print("Сводка по моделям на валидации:")
print(f"  Baseline (rmean_7):  wMAE = {baseline_wmae:.4f}")
print(f"  LightGBM:            wMAE = {weighted_mae(y_val, lgb_val_pred):.4f}")
print(f"  CatBoost:            wMAE = {weighted_mae(y_val, cb_val_pred):.4f}")
print(f"  Two-stage:           wMAE = {best_wmae_2s:.4f}")

# Grid-search весов для блендинга LGB + CB + Two-stage
best_blend_wmae = float("inf")
best_w = (1.0, 0.0, 0.0)

for w1 in np.arange(0, 1.05, 0.1):
    for w2 in np.arange(0, 1.05 - w1, 0.1):
        w3 = round(1.0 - w1 - w2, 2)
        if w3 < 0:
            continue
        blend = w1 * lgb_val_pred + w2 * cb_val_pred + w3 * two_stage_pred
        blend = np.clip(blend, 0, None)
        wm = weighted_mae(y_val, blend)
        if wm < best_blend_wmae:
            best_blend_wmae = wm
            best_w = (round(w1, 2), round(w2, 2), round(w3, 2))

print(f"\nЛучший ансамбль: LGB={best_w[0]}, CB={best_w[1]}, 2-stage={best_w[2]}")
print(f"  wMAE = {best_blend_wmae:.4f}")

In [ ]:
# ============================================================
# 5.6 Переобучение на ПОЛНОМ train (включая валидацию)
# ============================================================
# Модели выше обучены без последних 14 дней (валидация).
# Для финального предсказания переобучаем на всём train,
# используя лучшее число итераций, найденное на валидации.

train_all = df[(df["is_test"] == 0) & (df["dt"] >= df["dt"].min() + pd.Timedelta(days=28))].copy()
X_all = train_all[feature_cols]
y_all = train_all["qty"]
w_all = np.where(y_all > 0, 7.0, 1.0)

print(f"Переобучение на полном train: {len(X_all):,} строк")
print(f"  Период: {train_all['dt'].min().date()} — {train_all['dt'].max().date()}")

# LightGBM
print("\n[1/4] LightGBM...")
lgb_final = lgb.LGBMRegressor(**{**lgb_params, "n_estimators": lgb_model.best_iteration_})
lgb_final.fit(X_all, y_all, sample_weight=w_all)

# CatBoost
print("[2/4] CatBoost...")
cb_final = CatBoostRegressor(
    loss_function="MAE",
    iterations=cb_model.get_best_iteration(),
    learning_rate=0.05, depth=6, l2_leaf_reg=3.0,
    subsample=0.8, random_seed=42, verbose=0,
)
cb_final.fit(Pool(X_all, y_all, weight=w_all))

# Two-stage classifier
print("[3/4] Two-stage classifier...")
cls_final = lgb.LGBMClassifier(
    objective="binary", learning_rate=0.05, num_leaves=63,
    min_child_samples=30, subsample=0.8, colsample_bytree=0.8,
    n_estimators=max(cls_model.best_iteration_, 50),
    scale_pos_weight=7.0, random_state=42, verbose=-1, n_jobs=-1,
)
cls_final.fit(X_all, (y_all > 0).astype(int))

# Two-stage regressor
print("[4/4] Two-stage regressor...")
pos_mask = y_all > 0
reg_final = lgb.LGBMRegressor(
    objective="regression_l1", learning_rate=0.05, num_leaves=63,
    min_child_samples=20, subsample=0.8, colsample_bytree=0.8,
    n_estimators=max(reg_model.best_iteration_, 50),
    random_state=42, verbose=-1, n_jobs=-1,
)
reg_final.fit(X_all[pos_mask], y_all[pos_mask])

print("\nВсе модели переобучены на полном train.")

---
## 6. Предсказание и сабмит
- Переобученные модели на **полном train** (включая период валидации)
- Forward-fill лаговых фичей (вместо 93% NaN)
- Клиппинг: qty >= 0, qty <= prev_leftovers
- **БЕЗ округления** — дробные предсказания оптимальнее для MAE

In [ ]:
# ============================================================
# 6.1 Предсказание на test (переобученные модели, без рекурсии)
# ============================================================
test_df = df[df["is_test"] == 1].copy()
X_test = test_df[feature_cols]

# LightGBM (переобученный на полном train)
lgb_test_pred = lgb_final.predict(X_test)
lgb_test_pred = np.clip(lgb_test_pred, 0, None)

# CatBoost (переобученный на полном train)
cb_test_pred = cb_final.predict(X_test)
cb_test_pred = np.clip(cb_test_pred, 0, None)

# Two-stage (переобученный на полном train)
cls_test_proba = cls_final.predict_proba(X_test)[:, 1]
reg_test_pred = reg_final.predict(X_test)
reg_test_pred = np.clip(reg_test_pred, 0, None)
two_stage_test_pred = np.where(cls_test_proba >= best_thr, reg_test_pred, 0.0)

# Ансамбль с лучшими весами
w1, w2, w3 = best_w
final_pred = w1 * lgb_test_pred + w2 * cb_test_pred + w3 * two_stage_test_pred

print(f"Ансамбль: LGB*{w1} + CB*{w2} + 2-stage*{w3}")
print(f"Предсказания: min={final_pred.min():.3f}, max={final_pred.max():.3f}, mean={final_pred.mean():.3f}")

In [ ]:
# ============================================================
# 6.2 Постобработка
# ============================================================

# Клиппинг: не больше остатков на складе
test_leftovers = test_df["prev_leftovers"].values
final_pred = np.clip(final_pred, 0, test_leftovers)

# НЕ округляем — дробные предсказания дают лучший MAE
print(f"После постобработки:")
print(f"  min={final_pred.min():.3f}, max={final_pred.max():.3f}, mean={final_pred.mean():.3f}")
print(f"  Доля нулей: {(final_pred == 0).mean():.1%}")
print(f"  Доля > 0: {(final_pred > 0).mean():.1%}")

In [ ]:
# ============================================================
# 6.3 Формирование submission
# ============================================================

submission = sub.copy()
submission["qty"] = 0.0  # default (float, не int!)

# Маппинг предсказаний по (nm_id, dt)
test_df["pred_qty"] = final_pred
pred_map = test_df.set_index(["nm_id", "dt"])["pred_qty"]

submission["dt"] = pd.to_datetime(submission["dt"])
submission = submission.set_index(["nm_id", "dt"])
submission["qty"] = pred_map
submission = submission.reset_index()

# Заполняем NaN нулями (товары без предсказаний)
submission["qty"] = submission["qty"].fillna(0.0)

# Проверка
assert len(submission) == len(sub), f"Длина не совпадает: {len(submission)} vs {len(sub)}"
assert submission["qty"].isnull().sum() == 0, "Есть NaN в предсказаниях!"

print(f"Submission shape: {submission.shape}")
print(f"Пример:")
display(submission.head(10))

# Сохраняем
output_path = "/Users/slvic/Downloads/submission_2.csv"
submission.to_csv(output_path, index=False)
print(f"\nСохранено: {output_path}")

---
## Итоги

| Модель | wMAE (val) |
|--------|-----------|
| Baseline (rmean_7) | см. выше |
| LightGBM | см. выше |
| CatBoost | см. выше |
| Two-stage (cls + reg) | см. выше |
| **Ансамбль** | **лучший** |

**Улучшения (v3):**
1. **Forward-fill лаговых фичей** — вместо 93% NaN в тесте, лаги заполняются последними известными значениями
2. **Переобучение на полном train** — финальные модели обучены на всех данных (включая валидационный период)
3. **CatBoost с взвешенным eval** — early stopping через Pool с весами
4. **Без округления** — float-предсказания оптимальнее для MAE
5. **Индикатор устаревшости** — фича `days_since_train`

**Убрано (ухудшало):**
- ~~Клиппинг выбросов~~ — обрезал таргет на валидации, модель не могла предсказать высокие продажи
- ~~Рекурсивное предсказание~~ — накопление ошибок от дня к дню (mean рос с 1.08 до 1.99)